# Fluxonium Design with 6 single qubits Version 3.3

## Properties
1. LaunchPads are 200um width and height
2. Charge line has a bulp shape ending
3. JJ Array_width = 410um, JJ_width = 120um
4. L=200nH

## Chages from V3.2
1. Ec(chargin energy) 1.00 GHz, was 1.25 GHz
2. Resonators: 6.00-8.00 GHz, were 5-7 GHz

## Design and EPR analysis

### Preparations

In [1]:
# For convenience, let's begin by enabling
# automatic reloading of modules when they change.
%load_ext autoreload
%autoreload 2

import numpy as np
from collections import OrderedDict

import warnings
warnings.filterwarnings('ignore')

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict
import matplotlib.pyplot as plt

from qiskit_metal.toolbox_metal import math_and_overrides

from qiskit_metal.qlibrary.core import QComponent

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

from qiskit_metal.qlibrary.terminations.open_to_ground_v2 import OpenToGround
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer

from qiskit_metal.analyses.sweep_and_optimize.sweeping import Sweeping


from qiskit_metal.qlibrary.terminations.ebeam_marker_v2 import Markers
from qiskit_metal.qlibrary.terminations.ebeam_frame import Frame

import pyEPR as epr
from qiskit_metal.analyses.quantization import EPRanalysis

In [2]:
# Each time you create a new quantum circuit design,
# you start by instantiating a QDesign class.

# The design class `DesignPlanar` is best for 2D circuit designs.
design = designs.DesignPlanar()
design.chips.main.material = 'silicon'  # our substrate is Silicon
# cpw_width and cpw_gap is defined from the beginnning if you need you can 
# change it later. But with this way one can be sure all the lines are having same width and gap.
design.variables['cpw_width'] = '15 um'  
design.variables['cpw_gap'] = '8.733 um'  
design._chips['main']['size']['size_x'] = '9mm'  # Our device size is 9*9mm
design._chips['main']['size']['size_y'] = '9mm'

# If you disable the next line with "overwrite_enabled", then you will need to
# delete a component [<component>.delete()] before recreating it.
design.overwrite_enabled = True


eig_qres = EPRanalysis(design, "hfss")
hfss = design.renderers.hfss
hfss = eig_qres.sim.renderer
q3d = design.renderers.q3d

# run the GUI 
gui = MetalGUI(design)

In [3]:
from qiskit_metal.qlibrary.terminations.ebeam_marker_v2 import Markers
from qiskit_metal.qlibrary.terminations.ebeam_frame import Frame

## Markers for ebeam
options = dict(pos_x='3.25mm', pos_y='4.25mm')
mrk1 = Markers(design, 'marker1', options = options)

options = dict(pos_x='-3.25mm', pos_y='4.25mm')
mrk2 = Markers(design, 'marker2', options = options)

options = dict(pos_x='-3.25mm', pos_y='-4.25mm')
mrk3 = Markers(design, 'marker3', options = options)

options = dict(pos_x='3.25mm', pos_y='-4.25mm')
mrk4 = Markers(design, 'marker4', options = options)

options = dict(chip='main')
frame = Frame(design, 'frame', options = options)

gui.rebuild()

In [4]:
#Setup the launchpad1 location and orientation

## Read In launch pad
launch_options = dict(chip='main', pos_x='-4.0mm', pos_y='-3.99mm', orientation='0',
        lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPReadIn', options = launch_options)


# Read Out launch pad
launch_options = dict(chip='main', pos_x='4.0mm', pos_y='3.99mm', orientation='180',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPReadOut', options = launch_options)


Transmission_Line = {'pin_inputs':
           {'start_pin': {'component': 'LPReadIn', 'pin': 'tie'},
             'end_pin': {'component': 'LPReadOut', 'pin': 'tie'}},
            'lead': {'start_straight': '3970um', 'end_straight': '3970um',
                    # 'start_jogged_extension': jogs_in,
                    # 'anchors': anchors1,
                    }, 
                    'total_length': '8mm', 'fillet': "90um"
            }

TransLine = RoutePathfinder(design, 'TL', Transmission_Line)


gui.rebuild()

In [5]:
## Q1 flux-bias and charge line's lauch pad
launch_options = dict(chip='main', pos_x='-4.0mm', pos_y='-2.66mm', orientation='0',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPCL1', options = launch_options)

launch_options = dict(chip='main', pos_x='-4.0mm', pos_y='-1.33mm', orientation='0',
                     lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPFB1', options = launch_options)


## Q2 flux-bias and charge line's lauch pad
launch_options = dict(chip='main', pos_x='-4.0mm', pos_y='0.0mm', orientation='0',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPCL2', options = launch_options)

launch_options = dict(chip='main', pos_x='-4.0mm', pos_y='+1.33mm', orientation='0',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPFB2', options = launch_options)


## Q3 flux-bias and charge line's lauch pad
launch_options = dict(chip='main', pos_x='-4.0mm', pos_y='2.66mm', orientation='0',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPCL3', options = launch_options)

launch_options = dict(chip='main', pos_x='-4.0mm', pos_y='3.99mm', orientation='0',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPFB3', options = launch_options)


## Q4 flux-bias and charge line's lauch pad
launch_options = dict(chip='main', pos_x='4.0mm', pos_y='2.66mm', orientation='180',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPFB4', options = launch_options)

launch_options = dict(chip='main', pos_x='4.0mm', pos_y='1.33mm', orientation='180',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPCL4', options = launch_options)


## Q5 flux-bias and charge line's lauch pad
launch_options = dict(chip='main', pos_x='4.0mm', pos_y='0.0mm', orientation='180',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPFB5', options = launch_options)

launch_options = dict(chip='main', pos_x='4.0mm', pos_y='-1.33mm', orientation='180',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPCL5', options = launch_options)


## Q6 flux-bias and charge line's lauch pad
launch_options = dict(chip='main', pos_x='4.0mm', pos_y='-2.66mm', orientation='180',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPFB6', options = launch_options)

launch_options = dict(chip='main', pos_x='4.0mm', pos_y='-3.99mm', orientation='180',
                      lead_length='30um', pad_width='200um',
        pad_height='200um', pad_gap='112um',
                    )
lp = LaunchpadWirebond(design, 'LPCL6', options = launch_options)


gui.rebuild()

In [17]:
from qiskit_metal.qlibrary.qubits.fluxoniumEPR_v2 import FluxoniumPocket

gds_cell_name_jj = 'junction_0'
gds_cell_name_inductor = 'jj_array0'

# Q1 details
options = dict(chip='main', nanowire = False, 
            orientation =90, pos_x = '-2.2mm', pos_y = '-2.3mm',
            gds_cell_name=gds_cell_name_jj,
            gds_cell_inductor=gds_cell_name_inductor ,
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = True,
           ), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = True,
                            ),
    readout_line_options=Dict(loc_H = -1, make_rol = True, 
              ) )
               
q1 = FluxoniumPocket(design,'Q1', options = dict(
             **options))

# Q6 details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '2.2mm', pos_y = '-3.2mm',
        gds_cell_name=gds_cell_name_jj,
        gds_cell_inductor=gds_cell_name_inductor ,
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = True,
        ), 
               
    charge_line_options=Dict(loc_H = -1, make_cl = True,
                            ), 
    readout_line_options=Dict(loc_H = +1, make_rol = True,
              ) )
               
q6 = FluxoniumPocket(design,'Q6', options = dict(
        **options))


# Q2 details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '-2.2mm', pos_y = '0.0mm',
        gds_cell_name=gds_cell_name_jj,
        gds_cell_inductor=gds_cell_name_inductor ,
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = True,
         ), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = True,
                            ),
    readout_line_options=Dict(loc_H = -1, make_rol = True,
              ) )
               
q2 = FluxoniumPocket(design,'Q2', options = dict(
        **options))

# Q5 details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '2.4mm', pos_y = '-0.8mm',
        gds_cell_name=gds_cell_name_jj,
               gds_cell_inductor=gds_cell_name_inductor ,
               inductor_orientation='-1',
                      
    flux_bias_line_options=Dict(make_fbl = True,
                            ), 
               
    charge_line_options=Dict(loc_H = -1, make_cl = True,
                            ), 
    readout_line_options=Dict(loc_H = +1, make_rol = True,
              ) )
               
q5 = FluxoniumPocket(design,'Q5', options = dict(
        **options))


# Q3 details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '-2.5mm', pos_y = '3.0mm',
        gds_cell_name=gds_cell_name_jj,
        gds_cell_inductor=gds_cell_name_inductor ,
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = True,
        ), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = True,
                            ),
    readout_line_options=Dict(loc_H = -1, make_rol = True,
              ) )
               
q3 = FluxoniumPocket(design,'Q3', options = dict(
        **options))

# Q4 details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '2.4mm', pos_y = '2.0mm',
        gds_cell_name=gds_cell_name_jj,
        gds_cell_inductor=gds_cell_name_inductor,
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = True,
        ), 
               
    charge_line_options=Dict(loc_H = -1, make_cl = True,
                            ),
    readout_line_options=Dict(loc_H = 1, make_rol = True,
              ) )
               
q4 = FluxoniumPocket(design,'Q4', options = dict(
        **options))

# Q JJ1 Trial details
options = dict(chip='main',  nanowire = False,
        orientation =90, pos_x = '-1.0mm', pos_y = '1.2mm',
               pocket_width= '600um',
        gds_cell_name=gds_cell_name_jj,
               
    flux_bias_line_options=Dict(make_fbl = False), 
               
    charge_line_options=Dict(loc_H = -1, make_cl = False,
                            ),
    readout_line_options=Dict(loc_H = 1, make_rol = False,
              ) )
               
qJJ1 = FluxoniumPocket(design,'QJJ1', options = dict(
        **options))

# Q JJ2 Trial details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '-1.0mm', pos_y = '1.4mm',
               pocket_width= '600um',
        gds_cell_name=gds_cell_name_jj,   
               
    flux_bias_line_options=Dict(make_fbl = False), 
               
    charge_line_options=Dict(loc_H = -1, make_cl = False,
                            ),
    readout_line_options=Dict(loc_H = 1, make_rol = False,
              ) )
               
qJJ2 = FluxoniumPocket(design,'QJJ2', options = dict(
        **options))

# Q JJ3 Trial details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '-1.0mm', pos_y = '1.6mm',
               pocket_width= '600um',
        gds_cell_name=gds_cell_name_jj,  
               
    flux_bias_line_options=Dict(make_fbl = False), 
               
    charge_line_options=Dict(loc_H = -1, make_cl = False,
                            ),
    readout_line_options=Dict(loc_H = 1, make_rol = False,
              ) )
               
qJJ3 = FluxoniumPocket(design,'QJJ3', options = dict(
        **options))


# Q Array1 details
options = dict(chip='main', nanowire = False, 
            orientation =90, pos_x = '1.0mm', pos_y = '0.6mm',
            gds_cell_inductor=gds_cell_name_inductor , pocket_width= '600um',
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = False,
                        ), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = False,
               ),
    readout_line_options=Dict(loc_H = -1, make_rol = False, 
              ) )
               
qArr1 = FluxoniumPocket(design,'QArr1', options = dict(
             **options))


# Q Array2 Trial details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '1.0mm', pos_y = '0.8mm',
               pocket_width= '600um',
        gds_cell_inductor=gds_cell_name_inductor ,   
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = False,
                               ), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = False,
                            ),
    readout_line_options=Dict(loc_H = -1, make_rol = False, 
              ) )
               
qArr2 = FluxoniumPocket(design,'QArr2', options = dict(
        **options))

# Q Array3 Trial details
options = dict(chip='main', nanowire = False,
        orientation =90, pos_x = '1.0mm', pos_y = '1.0mm',
               pocket_width= '600um',
        gds_cell_inductor=gds_cell_name_inductor , 
               inductor_orientation='-1',
               
    flux_bias_line_options=Dict(make_fbl = False,
                      ), 
               
    charge_line_options=Dict(loc_H = +1, make_cl = False,
                            ),
    readout_line_options=Dict(loc_H = -1, make_rol = False, 
              ) )
               
qArr3 = FluxoniumPocket(design,'QArr3', options = dict(
        **options))


gui.rebuild()

## Creating the resonators
After creating Qubits and Launch pads you can create the resonators, we use lambda/2 resonators. For that you need to have open_to_ground 

In [18]:
# Lambda/2 resonators - Resonator1

otg1 = OpenToGround(design, 'otg1s', options=dict(chip='main', 
                 pos_x='-37.733um', pos_y='-2.8mm', orientation='270'))
rt_meander = RouteMeander(design, 'cavity1',  Dict(meander=Dict(spacing='200um'),
        total_length='7.4mm',
        hfss_wire_bonds = True,
        fillet='99um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
            start_pin=Dict(component='otg1s', pin='open'),
            end_pin=Dict(component='Q1', pin='readout_line')), ))

gui.rebuild()

In [19]:
# Lambda/2 resonators - Resonator6
otg6 = OpenToGround(design, 'otg6s', options=dict(chip='main', pos_x='37.733um',  pos_y='-2.8mm', orientation='270'))
rt_meander = RouteMeander(design, 'cavity6',  Dict(meander=Dict(spacing='200um'),
        total_length='7.8mm',
        hfss_wire_bonds = True,
        fillet='99um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
            start_pin=Dict(component='otg6s', pin='open'),
            end_pin=Dict(component='Q6', pin='readout_line')), ))

gui.rebuild()

In [20]:
# Lambda/2 resonators - Resonator2
otg2 = OpenToGround(design, 'otg2s', options=dict(chip='main', pos_x='-37.733um',  pos_y='-0.2mm', orientation='270'))
rt_meander = RouteMeander(design, 'cavity2',  Dict(meander=Dict(spacing='200um'),
        total_length='8.21mm',
        hfss_wire_bonds = True,
        fillet='99um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
            start_pin=Dict(component='otg2s', pin='open'),
            end_pin=Dict(component='Q2', pin='readout_line')), ))

gui.rebuild()

In [21]:
# Lambda/2 resonators - Resonator5
otg5 = OpenToGround(design, 'otg5s', options=dict(chip='main', pos_x='37.733um',  pos_y='-0.1mm', orientation='90'))
rt_meander = RouteMeander(design, 'cavity5',  Dict(meander=Dict(spacing='200um'),
        total_length='8.71mm',
        hfss_wire_bonds = True,
        fillet='99um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
            start_pin=Dict(component='otg5s', pin='open'),
            end_pin=Dict(component='Q5', pin='readout_line')), ))

gui.rebuild()

In [22]:
# Lambda/2 resonators - Resonator3
otg3 = OpenToGround(design, 'otg3s', options=dict(chip='main', pos_x='-37.733um',  pos_y='2.6mm', orientation='270'))
rt_meander = RouteMeander(design, 'cavity3',  Dict(meander=Dict(spacing='200um'),
        total_length='9.27mm',
        hfss_wire_bonds = True,
        fillet='99um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
            start_pin=Dict(component='otg3s', pin='open'),
            end_pin=Dict(component='Q3', pin='readout_line')), ))

gui.rebuild()

In [23]:
# Lambda/2 resonators - Resonator4
otg4 = OpenToGround(design, 'otg4s', options=dict(chip='main', pos_x='37.733um',  pos_y='2.6mm', orientation='90'))
rt_meander = RouteMeander(design, 'cavity4',  Dict(meander=Dict(spacing='200um'),
        total_length='9.82mm',
        hfss_wire_bonds = True,
        fillet='99um',
        lead = dict(start_straight='250um'),
        pin_inputs=Dict(
            start_pin=Dict(component='otg4s', pin='open'),
            end_pin=Dict(component='Q4', pin='readout_line')), ))


gui.rebuild()
gui.autoscale()

## Time to have the Charge and Flux-Bias lines

In [ ]:
# Charge Line for Q1

XYCharge_LineQ1 = RoutePathfinder(design, 'XY_Gate1', 
        options = dict(chip='main', fillet='99um',
            lead=dict(start_straight='150um', end_straight='600um', 
                      ),
                         pin_inputs=Dict( start_pin=Dict(
                                          component='Q1',
                                               pin='charge_line'),
                                                end_pin=Dict(
                                            component='LPCL1',
                                            pin='tie')
                                            )))

gui.rebuild()

In [ ]:
# Flux Bias Line for Q1

ZFluxBias_LineQ1 = RoutePathfinder(design, 'Z_Gate1', 
        options = dict(chip='main', fillet='99um',
            lead=dict(start_straight='200um', 
        end_straight='400um', ),
        pin_inputs=Dict(start_pin=Dict(component='Q1', 
              pin='flux_bias_line'),
          end_pin=Dict(component='LPFB1', pin='tie')
                                   )))

gui.rebuild()

In [ ]:
# Charge Lıne for Q2

XYCharge_LineQ2 = RoutePathfinder(design, 'XY_Gate2', options = dict(chip='main',
                        fillet='80um',
                lead=dict(start_straight='150um', end_straight='850um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='LPCL2',
                                                    pin='tie')
                                            )))

gui.rebuild()

In [ ]:
# Flux Bias Line for Q2

ZFluxBias_LineQ2 = RoutePathfinder(design, 'Z_Gate2', options = dict(chip='main',
             fillet='99um',
          lead=dict(start_straight='250um', end_straight='250um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='flux_bias_line'),
                                                end_pin=Dict(
                                                    component='LPFB2',
                                                    pin='tie')
                                            )))

gui.rebuild()

In [ ]:
# Charge Line for Q3

XYCharge_LineQ3 = RoutePathfinder(design, 'XY_Gate3', options = dict(chip='main',
                         fillet='99um',
       lead=dict(start_straight='250um', end_straight='500um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q3',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='LPCL3',
                                                    pin='tie')
                                            )))
gui.rebuild()

In [ ]:
# Flux Bias Line for Q3

ZFluxBias_LineQ3 = RoutePathfinder(design, 'Z_Gate3', options = dict(chip='main',
                                   fillet='99um',
           lead=dict(start_straight='100um', end_straight='200um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q3',
                                                    pin='flux_bias_line'),
                                                end_pin=Dict(
                                                    component='LPFB3',
                                                    pin='tie')
                                            )))

gui.rebuild()

In [ ]:
# Charge Lıne for Q6

XYCharge_LineQ6 = RoutePathfinder(design, 'XY_Gate6', 
        options = dict(chip='main', fillet='99um',
        lead=dict(start_straight='100um', end_straight='250um'),
    pin_inputs=Dict(start_pin=Dict(component='Q6', pin='charge_line'),
    end_pin=Dict(component='LPCL6', pin='tie')
                                            )))

gui.rebuild()

In [ ]:
# Flux Bias Line for Q6
ZFluxBias_LineQ6 = RoutePathfinder(design, 'Z_Gate6', 
        options = dict(chip='main', fillet='99um',
            lead=dict(start_straight='350um', end_straight='350um'),
        pin_inputs=Dict(start_pin=Dict(component='Q6', 
              pin='flux_bias_line'),
          end_pin=Dict(component='LPFB6', pin='tie')
                                   )))

gui.rebuild()

In [ ]:
# Charge Lıne for Q5

XYCharge_LineQ5 = RoutePathfinder(design, 'XY_Gate5', options = dict(chip='main',
                          fillet='99um',
            lead=dict(start_straight='150um', end_straight='850um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q5',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='LPCL5',
                                                    pin='tie')
                                            )))

gui.rebuild()

In [ ]:
# Flux Bias Line for Q5

ZFluxBias_LineQ5 = RoutePathfinder(design, 'Z_Gate5', options = dict(chip='main',
                        fillet='99um',
            lead=dict(start_straight='200um', end_straight='450um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q5',
                                                    pin='flux_bias_line'),
                                                end_pin=Dict(
                                                    component='LPFB5',
                                                    pin='tie')
                                            )))
gui.rebuild()

In [ ]:
# Charge Line for Q4

XYCharge_LineQ4 = RoutePathfinder(design, 'XY_Gate4', options = dict(chip='main',
                                 fillet='99um',
                    lead=dict(start_straight='200um', end_straight='550um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q4',
                                                    pin='charge_line'),
                                                end_pin=Dict(
                                                    component='LPCL4',
                                                    pin='tie')
                                            )))
gui.rebuild()

In [ ]:
# Flux Bias Line for Q4

ZFluxBias_LineQ4 = RoutePathfinder(design, 'Z_Gate4', options = dict(chip='main',
                                            fillet='99um',
                    lead=dict(start_straight='150um', end_straight='200um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q4',
                                                    pin='flux_bias_line'),
                                                end_pin=Dict(
                                                    component='LPFB4',
                                                    pin='tie')
                                            )))
gui.rebuild()

In [ ]:
q1 #print Q1 information


In [ ]:
#Save screenshot as a .png formatted file.
gui.screenshot()

## Ansys QRenderer
How to Render a Metal Design into Ansys

In [24]:
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer
QAnsysRenderer.default_options

{'Lj': '10nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'x_buffer_width_mm': 0.2,
 'y_buffer_width_mm': 0.2,
 'wb_threshold': '400um',
 'wb_offset': '0um',
 'wb_size': 5,
 'plot_ansys_fields_options': {'name': 'NAME:Mag_E1',
  'UserSpecifyName': '0',
  'UserSpecifyFolder': '0',
  'QuantityName': 'Mag_E',
  'PlotFolder': 'E Field',
  'StreamlinePlot': 'False',
  'AdjacentSidePlot': 'False',
  'FullModelPlot': 'False',
  'IntrinsicVar': "Phase='0deg'",
  'PlotGeomInfo_0': '1',
  'PlotGeomInfo_1': 'Surface',
  'PlotGeomInfo_2': 'FacesList',
  'PlotGeomInfo_3': '1'}}

!!!Make sure the software is open completly.

In [ ]:
## Make sure the software is open completly then run the next cell.
hfss.open_ansys()

In [ ]:
hfss.connect_ansys()

First, we will simulate every resonator one by one to get their frequencies.

In [ ]:
hfss.activate_eigenmode_design("cavity1")

In [ ]:
hfss.render_design(['cavity1'], [('cavity1', 'end'), ('cavity1', 'start')])

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 15
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.epr_get_frequencies()

In [ ]:
hfss.activate_eigenmode_design("cavity6")

In [ ]:
hfss.render_design(['cavity6'], [('cavity6', 'end'), ('cavity6', 'start')])

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 15
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.epr_get_frequencies()

In [ ]:
hfss.activate_eigenmode_design("cavity2")

In [ ]:
hfss.render_design(['cavity2'], [('cavity2', 'end'), ('cavity2', 'start')])

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 15
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.save_screenshot()

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.epr_get_frequencies()

In [ ]:
hfss.activate_eigenmode_design("cavity5")

In [ ]:
hfss.render_design(['cavity5'], [('cavity5', 'end'), ('cavity5', 'start')])

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 15
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.epr_get_frequencies()

In [ ]:
hfss.activate_eigenmode_design("cavity3")

In [ ]:
hfss.render_design(['cavity3'], [('cavity3', 'end'), ('cavity3', 'start')])

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 15
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.epr_get_frequencies()

In [ ]:
hfss.activate_eigenmode_design("cavity4")

In [ ]:
hfss.render_design(['cavity4'], [('cavity4', 'end'), ('cavity4', 'start')])

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.passes = 15
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.epr_get_frequencies()

In [ ]:
QAnsysRenderer.get_template_options(design)

In [ ]:
QAnsysRenderer.default_setup

In [ ]:
# Run this cell only if you are done with ANSYS
hfss.disconnect_ansys()

## Now, we can use 2 eigenmodes to get the resonator and the qubit frequencies.
# Before one could get the BARE frequencies of a resonator, after this step you will get the DRESSED frequency of the resonator

In [ ]:
hfss.activate_eigenmode_design("Q1_Res1")

In [ ]:
hfss.options['x_buffer_width_mm'] = 0.9
hfss.options['y_buffer_width_mm'] = 0.9
hfss.options

In [ ]:
design.qgeometry.tables['junction']

In [ ]:
hfss.render_design(['Q1', 'cavity1', 'otg1s'],[('cavity1', 'end')])

In [ ]:
hfss.save_screenshot()

In [ ]:
# Analysis properties
setup = hfss.pinfo.setup
setup.n_modes = 2
setup.passes = 15
#setup.delta_f = 0.05
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '34.38 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

In [ ]:
hfss.set_mode(2, "Setup")

In [ ]:
hfss.modeler._modeler.ShowWindow()
hfss.plot_ansys_fields('main')
hfss.save_screenshot()

In [ ]:
hfss.save_screenshot()

In [ ]:
hfss.plot_ansys_delete(['Mag_E1'])

# Finite Element Analysis with EPR

After this point, you have to use the EPR version for Fluxonium and you need to have a new environment for that.
You can use that environment from the beginning too. 

In [25]:
eig_qb = EPRanalysis(design,"hfss")

# Q1 and Res 1 

In [26]:
eig_qb.sim.setup_update(n_modes = 2,
                        max_delta_f = 0.1,
                        min_freq_ghz = 1,
                        max_passes = 12)
eig_qb.sim.setup.vars.Lj = '34.3nH' # targetted E_j=4.75GHz
eig_qb.sim.setup.vars.Cj = '0.0nH'
eig_qb.sim.setup.vars.Ls = q1.options.l_inductance

eig_qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 2,
 'max_delta_f': 0.1,
 'max_passes': 12,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '34.3nH', 'Cj': '0.0nH', 'Ls': '200nH'}}

In [27]:
eig_qb.sim.run(name="FluxoniumEPR", components=['Q1', 'cavity1', 'otg1s'], open_terminations=[], box_plus_buffer = True)

INFO 09:00PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:00PM [load_ansys_project]: 	Opened Ansys App
INFO 09:00PM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 09:00PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/figenyilmaz/Documents/Ansoft/
	Project:   Project49
INFO 09:00PM [connect_design]: No active design found (or error getting active design).
INFO 09:00PM [connect]: 	 Connected to project "Project49". No design detected
INFO 09:00PM [connect_design]: 	Opened active design
	Design:    FluxoniumEPR_hfss [Solution type: Eigenmode]
WARNING 09:00PM [connect_setup]: 	No design setup detected.
WARNING 09:00PM [connect_setup]: 	Creating eigenmode default setup.
INFO 09:00PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:00PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 09:00PM [analyze]: Analyzing setup Setup
09:11PM 57s INFO [get_f_convergence]: Saved convergences to C:

In [28]:
eig_qb.sim.convergence_f

,re(Mode(1)) [g],re(Mode(2)) [g]
Pass [],,
1,3.950215,5.339629
2,4.707301,5.716074
3,5.112197,5.889477
4,5.668083,6.422763
5,5.894121,6.763241
6,6.011019,6.886078
7,6.131316,6.954929
8,6.169188,7.005250
9,6.184896,7.027050


### Run EPR analysis

Perform EPR analysis for all modes and variations. Here, Quantum Analysis starts.

In [38]:
# Add the geometry of the rectangle  and line of the inductor to the list of junctions
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q1_rect_jj_'

eig_qb.add_junction(name_junction ='inductor')
eig_qb.setup.junctions.inductor.Lj_variable = 'Ls'
eig_qb.setup.junctions.inductor.rect = 'JJ_rect_Lj_Q1_inductor'
eig_qb.setup.junctions.inductor.line = 'JJ_Lj_Q1_inductor_'
# eig_qb.setup.junctions.inductor.is_junction = False

# Set up th EPR analysis parameters
eig_qb.setup.fock_trunc = 20
eig_qb.setup.cos_trunc = 6
eig_qb.setup.flux = 0.0

eig_qb.setup.basis = 'HO'

# eig_qb.setup.sweep_variable = 'flux'   #--- This would be nice to implement to get a sweep over flux bias 
eig_qb.setup

09:52PM 12s WARNING [add_junction]: junction already defined. Overwriting inductor


{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q1_rect_jj',
   'line': 'JJ_Lj_Q1_rect_jj_'},
  'inductor': {'Lj_variable': 'Ls',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_Q1_inductor',
   'line': 'JJ_Lj_Q1_inductor_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 6,
 'fock_trunc': 20,
 'flux': 0.0,
 'basis': 'HO',
 'sweep_variable': 'Lj'}

In [39]:
# eig_qb.run_epr()
#### equivalent individual calls
s = eig_qb.setup
eig_qb.epr_start()
eig_qb.get_stored_energy()
eig_qb.run_analysis()

Design "FluxoniumEPR_hfss" info:
	# eigenmodes    2
	# variations    1
Design "FluxoniumEPR_hfss" info:
	# eigenmodes    2
	# variations    1
Design "FluxoniumEPR_hfss" info:
	# eigenmodes    2
	# variations    1

        energy_elec_all       = 5.46086249772797e-24
        energy_elec_substrate = 5.02284224832369e-24
        EPR of substrate = 92.0%

        energy_mag    = 5.42843300348447e-24
        energy_mag % of energy_elec_all  = 99.4%
        

Variation 0  [1/1]

  Mode 0 at 6.30 GHz   [1/2]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.9%  1.331e-25 1.496e-27

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 90.29%
	jj              0.845082  (+)        0.0909107
		Energy fraction (Lj over Lj&Cj)= 61.45%
	inductor        0.145233  (+)        0.0910996
		(U_tot_cap-U_tot_ind)/mean=8.26%


In [40]:
eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
eig_qb.report_hamiltonian(s.sweep_variable)

WARNING 09:53PM [__init__]: <p>Error: <class 'IndexError'></p>
ERROR 09:53PM [_get_participation_normalized]: WARNING: U_tot_cap-U_tot_ind / mean = 16.5% is > 15%.                     
Is the simulation converged? Proceed with caution


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

[3.43e-08 2.00e-07]
[0]
Using Full Cosine Potential
ϕzpf mode 0 :  -0.7473070198755349
ϕzpf mode 1 :  0.06145481659541618
Starting the diagonalization
Finished the diagonalization
(2, 1)
Single junctions -- assuming single qubit mode


ERROR 09:53PM [_get_participation_normalized]: WARNING: U_tot_cap-U_tot_ind / mean = 16.5% is > 15%.                     
Is the simulation converged? Proceed with caution


Pm_norm=
modes
0    1.181288
1    1.112367
dtype: float64

Pm_norm idx =
      jj  inductor
0   True     False
1  False     False
*** P (participation matrix, not normlz.)
         jj  inductor
0  0.714953  0.122869
1  0.005098  0.000876

*** S (sign-bit matrix)
   s_jj  s_inductor
0    -1          -1
1     1           1
*** P (participation matrix, normalized.)
      0.84
    0.0051

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
       743     10.1
      10.1    0.034

*** Chi matrix ND (MHz) 
  1.12e+03     2.27
      2.27  0.00121

*** Frequencies O1 PT (MHz)
0    5554.352158
1    7055.328616
dtype: float64

*** Frequencies ND (MHz)
0    5448.094838
1    7057.676066
dtype: float64

*** Q_coupling
Empty DataFrame
Columns: []
Index: [0, 1]


#### Mode frequencies (MHz)

###### Numerical diagonalization

Lj,34.3
0,5448.09
1,7057.68


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0         1
Lj                       
34.3 0  1123.82  2.27e+00
     1     2.27  1.21e-03

# Loop over flux

In [41]:
from IPython.display import clear_output
chis = []
freq_q = []
freq_r = []
phis = np.linspace(0,1,201)
for phi in phis:
    eig_qb.setup.flux = phi
    s = eig_qb.setup
    eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
    a = eig_qb.sim.renderer.epr_quantum_analysis.get_frequencies()
    freq_q.append(a['0'][1])
    freq_r.append(a['0'][0])
    b = eig_qb.sim.renderer.epr_quantum_analysis.get_chis()
    chi = -b[0][1]
    chis.append(chi)
    clear_output()

In [42]:
%matplotlib inline

In [43]:
plt.plot(phis,np.array(freq_r)/1000)
plt.plot(phis,np.array(freq_q)/1000)
plt.xlabel(r'Flux, $\phi/\pi$')
plt.ylabel('frequencies, f (GHz)')
plt.ylim(0,10)

(0.0, 10.0)

In [44]:
plt.plot(phis,chis)
#plt.xlabel(r'Flux, $\phi/\pi$')
plt.xlabel(r'External Flux, $\Phi_{ext}/\Phi_0$')
plt.ylabel(r'Dispersive Shift, $\chi/2\pi$ (MHz)')
#plt.ylabel(r'Dispersive shift, $\chi$ (MHz)')

Text(523.2569444444446, 0.5, 'Dispersive Shift, $\\chi/2\\pi$ (MHz)')

# Q2 and Res 2 

In [ ]:
eig_qb.sim.setup_update(n_modes = 2,
                        max_delta_f = 0.1,
                        min_freq_ghz = 1,
                        max_passes = 12)
eig_qb.sim.setup.vars.Lj = '34.3nH' # targetted E_j=4.75GHz
eig_qb.sim.setup.vars.Cj = '0.0nH'
eig_qb.sim.setup.vars.Ls = q1.options.l_inductance

eig_qb.sim.setup

In [ ]:
eig_qb.sim.run(name="FluxoniumEPR", components=['Q2', 'cavity2', 'otg2s'], open_terminations=[], box_plus_buffer = True)

In [ ]:
eig_qb.sim.convergence_f

In [ ]:
# Add the geometry of the rectangle  and line of the inductor to the list of junctions
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q2_rect_jj_'

eig_qb.add_junction(name_junction ='inductor')
eig_qb.setup.junctions.inductor.Lj_variable = 'Ls'
eig_qb.setup.junctions.inductor.rect = 'JJ_rect_Lj_Q2_inductor'
eig_qb.setup.junctions.inductor.line = 'JJ_Lj_Q2_inductor_'
# eig_qb.setup.junctions.inductor.is_junction = False

# Set up th EPR analysis parameters
eig_qb.setup.fock_trunc = 15
eig_qb.setup.cos_trunc = 6
eig_qb.setup.flux = 0.0

eig_qb.setup.basis = 'HO'

# eig_qb.setup.sweep_variable = 'flux'   #--- This would be nice to implement to get a sweep over flux bias 
eig_qb.setup

In [ ]:
# eig_qb.run_epr()
#### equivalent individual calls
s = eig_qb.setup
eig_qb.epr_start()
eig_qb.get_stored_energy()
eig_qb.run_analysis()

In [ ]:
eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
eig_qb.report_hamiltonian(s.sweep_variable)

# Loop over flux, Q2

In [ ]:
from IPython.display import clear_output
chis = []
freq_q = []
freq_r = []
phis = np.linspace(0,1,301)
for phi in phis:
    eig_qb.setup.flux = phi
    s = eig_qb.setup
    eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
    a = eig_qb.sim.renderer.epr_quantum_analysis.get_frequencies()
    freq_q.append(a['0'][0])
    freq_r.append(a['0'][1])
    b = eig_qb.sim.renderer.epr_quantum_analysis.get_chis()
    chi = -b[0][1]
    chis.append(chi)
    clear_output()

In [ ]:
plt.plot(phis,np.array(freq_r)/1000)
plt.plot(phis,np.array(freq_q)/1000)
plt.xlabel(r'Flux, $\phi/\pi$')
plt.ylabel('frequencies, f (GHz)')
plt.ylim(0,10)

In [ ]:
plt.plot(phis,chis)
#plt.xlabel(r'Flux, $\phi/\pi$')
plt.xlabel(r'External Flux, $\Phi_{ext}/\Phi_0$')
plt.ylabel(r'Dispersive Shift, $\chi/2\pi$ (MHz)')
#plt.ylabel(r'Dispersive shift, $\chi$ (MHz)')

# Q3 and Res 3

In [ ]:
eig_qb.sim.setup_update(n_modes = 2,
                        max_delta_f = 0.1,
                        min_freq_ghz = 1,
                        max_passes = 12)
eig_qb.sim.setup.vars.Lj = '34.3nH' # targetted E_j=4.75GHz
eig_qb.sim.setup.vars.Cj = '0.0nH'
eig_qb.sim.setup.vars.Ls = q1.options.l_inductance

eig_qb.sim.setup

In [ ]:
eig_qb.sim.run(name="FluxoniumEPR", components=['Q3', 'cavity3', 'otg3s'], open_terminations=[], box_plus_buffer = True)

In [ ]:
eig_qb.sim.convergence_f

In [ ]:
# Add the geometry of the rectangle  and line of the inductor to the list of junctions
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q3_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q3_rect_jj_'

eig_qb.add_junction(name_junction ='inductor')
eig_qb.setup.junctions.inductor.Lj_variable = 'Ls'
eig_qb.setup.junctions.inductor.rect = 'JJ_rect_Lj_Q3_inductor'
eig_qb.setup.junctions.inductor.line = 'JJ_Lj_Q3_inductor_'
# eig_qb.setup.junctions.inductor.is_junction = False

# Set up th EPR analysis parameters
eig_qb.setup.fock_trunc = 15
eig_qb.setup.cos_trunc = 6
eig_qb.setup.flux = 0.0

eig_qb.setup.basis = 'HO'

# eig_qb.setup.sweep_variable = 'flux'   #--- This would be nice to implement to get a sweep over flux bias 
eig_qb.setup

In [ ]:
# eig_qb.run_epr()
#### equivalent individual calls
s = eig_qb.setup
eig_qb.epr_start()
eig_qb.get_stored_energy()
eig_qb.run_analysis()

In [ ]:
eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
eig_qb.report_hamiltonian(s.sweep_variable)

# Loop over flux, Q3

In [ ]:
from IPython.display import clear_output
chis = []
freq_q = []
freq_r = []
phis = np.linspace(0,1,301)
for phi in phis:
    eig_qb.setup.flux = phi
    s = eig_qb.setup
    eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
    a = eig_qb.sim.renderer.epr_quantum_analysis.get_frequencies()
    freq_q.append(a['0'][0])
    freq_r.append(a['0'][1])
    b = eig_qb.sim.renderer.epr_quantum_analysis.get_chis()
    chi = -b[0][1]
    chis.append(chi)
    clear_output()

In [ ]:
plt.plot(phis,np.array(freq_r)/1000)
plt.plot(phis,np.array(freq_q)/1000)
plt.xlabel(r'Flux, $\phi/\pi$')
plt.ylabel('frequencies, f (GHz)')
plt.ylim(0,10)

In [ ]:
plt.plot(phis,chis)
#plt.xlabel(r'Flux, $\phi/\pi$')
plt.xlabel(r'External Flux, $\Phi_{ext}/\Phi_0$')
plt.ylabel(r'Dispersive Shift, $\chi/2\pi$ (MHz)')
#plt.ylabel(r'Dispersive shift, $\chi$ (MHz)')

# Q4 and Res 4

In [ ]:
eig_qb.sim.setup_update(n_modes = 2,
                        max_delta_f = 0.1,
                        min_freq_ghz = 1,
                        max_passes = 12)
eig_qb.sim.setup.vars.Lj = '34.3nH' # targetted E_j=4.75GHz
eig_qb.sim.setup.vars.Cj = '0.0nH'
eig_qb.sim.setup.vars.Ls = q1.options.l_inductance

eig_qb.sim.setup

In [ ]:
eig_qb.sim.run(name="FluxoniumEPR", components=['Q4', 'cavity4', 'otg4s'], open_terminations=[], box_plus_buffer = True)

In [ ]:
eig_qb.sim.convergence_f

In [ ]:
# Add the geometry of the rectangle  and line of the inductor to the list of junctions
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q4_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q4_rect_jj_'

eig_qb.add_junction(name_junction ='inductor')
eig_qb.setup.junctions.inductor.Lj_variable = 'Ls'
eig_qb.setup.junctions.inductor.rect = 'JJ_rect_Lj_Q4_inductor'
eig_qb.setup.junctions.inductor.line = 'JJ_Lj_Q4_inductor_'
# eig_qb.setup.junctions.inductor.is_junction = False

# Set up th EPR analysis parameters
eig_qb.setup.fock_trunc = 15
eig_qb.setup.cos_trunc = 6
eig_qb.setup.flux = 0.0

eig_qb.setup.basis = 'HO'

# eig_qb.setup.sweep_variable = 'flux'   #--- This would be nice to implement to get a sweep over flux bias 
eig_qb.setup

In [ ]:
# eig_qb.run_epr()
#### equivalent individual calls
s = eig_qb.setup
eig_qb.epr_start()
eig_qb.get_stored_energy()
eig_qb.run_analysis()

In [ ]:
eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
eig_qb.report_hamiltonian(s.sweep_variable)

# Loop over flux, Q4

In [ ]:
from IPython.display import clear_output
chis = []
freq_q = []
freq_r = []
phis = np.linspace(0,1,301)
for phi in phis:
    eig_qb.setup.flux = phi
    s = eig_qb.setup
    eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
    a = eig_qb.sim.renderer.epr_quantum_analysis.get_frequencies()
    freq_q.append(a['0'][1])
    freq_r.append(a['0'][0])
    b = eig_qb.sim.renderer.epr_quantum_analysis.get_chis()
    chi = -b[0][1]
    chis.append(chi)
    clear_output()

In [ ]:
plt.plot(phis,np.array(freq_r)/1000)
plt.plot(phis,np.array(freq_q)/1000)
plt.xlabel(r'Flux, $\phi/\pi$')
plt.ylabel('frequencies, f (GHz)')
plt.ylim(0,10)

In [ ]:
plt.plot(phis,chis)
#plt.xlabel(r'Flux, $\phi/\pi$')
plt.xlabel(r'External Flux, $\Phi_{ext}/\Phi_0$')
plt.ylabel(r'Dispersive Shift, $\chi/2\pi$ (MHz)')
#plt.ylabel(r'Dispersive shift, $\chi$ (MHz)')

# Q5 and Res 5

In [ ]:
eig_qb.sim.setup_update(n_modes = 2,
                        max_delta_f = 0.1,
                        min_freq_ghz = 1,
                        max_passes = 12)
eig_qb.sim.setup.vars.Lj = '34.3nH' # targetted E_j=4.75GHz
eig_qb.sim.setup.vars.Cj = '0.0nH'
eig_qb.sim.setup.vars.Ls = q1.options.l_inductance

eig_qb.sim.setup

In [ ]:
eig_qb.sim.run(name="FluxoniumEPR", components=['Q5', 'cavity5', 'otg5s'], open_terminations=[], box_plus_buffer = False)

In [ ]:
eig_qb.sim.convergence_f

In [ ]:
# Add the geometry of the rectangle  and line of the inductor to the list of junctions
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q5_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q5_rect_jj_'

eig_qb.add_junction(name_junction ='inductor')
eig_qb.setup.junctions.inductor.Lj_variable = 'Ls'
eig_qb.setup.junctions.inductor.rect = 'JJ_rect_Lj_Q5_inductor'
eig_qb.setup.junctions.inductor.line = 'JJ_Lj_Q5_inductor_'
# eig_qb.setup.junctions.inductor.is_junction = False

# Set up th EPR analysis parameters
eig_qb.setup.fock_trunc = 15
eig_qb.setup.cos_trunc = 6
eig_qb.setup.flux = 0.0

eig_qb.setup.basis = 'HO'

# eig_qb.setup.sweep_variable = 'flux'   #--- This would be nice to implement to get a sweep over flux bias 
eig_qb.setup

In [ ]:
# eig_qb.run_epr()
#### equivalent individual calls
s = eig_qb.setup
eig_qb.epr_start()
eig_qb.get_stored_energy()
eig_qb.run_analysis()

In [ ]:
eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
eig_qb.report_hamiltonian(s.sweep_variable)

# Loop over flux, Q5

In [ ]:
from IPython.display import clear_output
chis = []
freq_q = []
freq_r = []
phis = np.linspace(0,1,301)
for phi in phis:
    eig_qb.setup.flux = phi
    s = eig_qb.setup
    eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
    a = eig_qb.sim.renderer.epr_quantum_analysis.get_frequencies()
    freq_q.append(a['0'][1])
    freq_r.append(a['0'][0])
    b = eig_qb.sim.renderer.epr_quantum_analysis.get_chis()
    chi = -b[0][1]
    chis.append(chi)
    clear_output()

In [ ]:
plt.plot(phis,np.array(freq_r)/1000)
plt.plot(phis,np.array(freq_q)/1000)
plt.xlabel(r'Flux, $\phi/\pi$')
plt.ylabel('frequencies, f (GHz)')
plt.ylim(0,10)

In [ ]:
plt.plot(phis,chis)
#plt.xlabel(r'Flux, $\phi/\pi$')
plt.xlabel(r'External Flux, $\Phi_{ext}/\Phi_0$')
plt.ylabel(r'Dispersive Shift, $\chi/2\pi$ (MHz)')
#plt.ylabel(r'Dispersive shift, $\chi$ (MHz)')

# Q6 and Res 6

In [ ]:
eig_qb.sim.setup_update(n_modes = 2,
                        max_delta_f = 0.1,
                        min_freq_ghz = 1,
                        max_passes = 12)
eig_qb.sim.setup.vars.Lj = '34.3nH' # targetted E_j=4.75GHz
eig_qb.sim.setup.vars.Cj = '0.0nH'
eig_qb.sim.setup.vars.Ls = q1.options.l_inductance

eig_qb.sim.setup

In [ ]:
eig_qb.sim.run(name="FluxoniumEPR", components=['Q6', 'cavity6', 'otg6s'], open_terminations=[], box_plus_buffer = True)

In [ ]:
eig_qb.sim.convergence_f

In [ ]:
# Add the geometry of the rectangle  and line of the inductor to the list of junctions
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_Q6_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_Q6_rect_jj_'

eig_qb.add_junction(name_junction ='inductor')
eig_qb.setup.junctions.inductor.Lj_variable = 'Ls'
eig_qb.setup.junctions.inductor.rect = 'JJ_rect_Lj_Q6_inductor'
eig_qb.setup.junctions.inductor.line = 'JJ_Lj_Q6_inductor_'
# eig_qb.setup.junctions.inductor.is_junction = False

# Set up th EPR analysis parameters
eig_qb.setup.fock_trunc = 15
eig_qb.setup.cos_trunc = 6
eig_qb.setup.flux = 0.0

eig_qb.setup.basis = 'HO'

# eig_qb.setup.sweep_variable = 'flux'   #--- This would be nice to implement to get a sweep over flux bias 
eig_qb.setup

In [ ]:
# eig_qb.run_epr()
#### equivalent individual calls
s = eig_qb.setup
eig_qb.epr_start()
eig_qb.get_stored_energy()
eig_qb.run_analysis()

In [ ]:
eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
eig_qb.report_hamiltonian(s.sweep_variable)

# Loop over flux, Q6

In [ ]:
from IPython.display import clear_output
chis = []
freq_q = []
freq_r = []
phis = np.linspace(0,1,301)
for phi in phis:
    eig_qb.setup.flux = phi
    s = eig_qb.setup
    eig_qb.spectrum_analysis(s.cos_trunc, s.fock_trunc, basis=s.basis, flux=s.flux, junctions=[0])
    a = eig_qb.sim.renderer.epr_quantum_analysis.get_frequencies()
    freq_q.append(a['0'][1])
    freq_r.append(a['0'][0])
    b = eig_qb.sim.renderer.epr_quantum_analysis.get_chis()
    chi = -b[0][1]
    chis.append(chi)
    clear_output()

In [ ]:
plt.plot(phis,np.array(freq_r)/1000)
plt.plot(phis,np.array(freq_q)/1000)
plt.xlabel(r'Flux, $\phi/\pi$')
plt.ylabel('frequencies, f (GHz)')
plt.ylim(0,10)

In [ ]:
plt.plot(phis,chis)
#plt.xlabel(r'Flux, $\phi/\pi$')
plt.xlabel(r'External Flux, $\Phi_{ext}/\Phi_0$')
plt.ylabel(r'Dispersive Shift, $\chi/2\pi$ (MHz)')
#plt.ylabel(r'Dispersive shift, $\chi$ (MHz)')

In [ ]:
hfss.disconnect_ansys()

In [ ]:
gui.main_window.close()